# Python for Medicine: Scrub-In Systems Check

Before we dive into data, we make sure the AI diagnostics cart boots without surprises. Today you will scrub in as the systems lead who keeps the ward's tooling sterile.

## Dawn Briefing

**Patient:** Jordan Nguyen (54, they/them) admitted with community-acquired pneumonia complicated by sepsis.  
**Today's Role:** Junior doctor assigned to certify that the MedVision learning rig is ready before rounds begin.

**Learning objectives**
- Understand how Python runtimes, packages, and paths fit together in Google Colab and on a local machine.
- Practise installing, inspecting, and verifying dependencies without peeking behind the helper-library curtain.
- Build a troubleshooting checklist so you can diagnose setup alarms before they disrupt patient care.

Plan for ~40 minutes of focused prep. Complete this notebook before starting `01_python_first_dose.ipynb`.

### How to Use This Notebook
- Execute cells in order. If output looks suspicious, restart (`Runtime → Restart and run all`) to ensure a clean environment.
- Challenge cells raise `NotImplementedError` until you supply an answer—this is intentional so problems surface early.
- Solution cells stay collapsed. Attempt the exercise honestly before expanding.
- Treat every warning as paging the med tech team. Investigate, document, and verify before continuing.

## 0. Prep the Bay: System Setup

This single cell provisions everything Project MedVision needs. It clones the repository when running in Colab, installs Python packages, and wires the helper library onto the path. If any step fails, the cell stops immediately so you can investigate.

In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path
from typing import Final

REPO_URL: Final[str] = "https://github.com/BradSegal/OxfordADH2025PythonBootcamp.git"

workspace = Path.cwd()

if (workspace / "src" / "medvision_toolkit").exists():
    repo_root = workspace
    print("[OK] Detected Project MedVision source in the current workspace.")
elif (workspace.parent / "src" / "medvision_toolkit").exists():
    repo_root = workspace.parent
    print("[OK] Detected Project MedVision source in parent directory.")
    print(f"  Project root: {repo_root}")
else:
    repo_root = workspace / "python-for-medicine-bootcamp"
    if repo_root.exists():
        print("Existing checkout found. Pulling latest changes…")
        result = subprocess.run(
            ["git", "-C", str(repo_root), "pull", "--ff-only"],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git pull failed; please resolve and rerun the cell.")
    else:
        print(f"Cloning repository from {REPO_URL} …")
        result = subprocess.run(
            ["git", "clone", "--depth", "1", REPO_URL, str(repo_root)],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print(result.stdout)
            print(result.stderr, file=sys.stderr)
            raise RuntimeError("git clone failed; check the URL or your connection.")

requirements_path = repo_root / "requirements.txt"
if not requirements_path.exists():
    raise FileNotFoundError(f"Could not locate requirements.txt at {requirements_path}")

try:
    import google.colab  # type: ignore

    in_colab = True
except ImportError:
    in_colab = False

if in_colab:
    print("Installing Python dependencies (this may take a minute)…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_path)])
else:
    print("Running locally. Assuming dependencies are already installed.")
    print("If you encounter import errors, run: pip install -r requirements.txt")

project_src = repo_root / "src"
if str(project_src) not in sys.path:
    sys.path.insert(0, str(project_src))
    print(f"[OK] Added {project_src} to Python path")

print("Environment ready. Proceed to the next cell.")

### Quick Systems Check
If the toolkit loads without complaint, your environment is ready for clinical exercises. Treat any exception as a red-alert page and document what you changed while fixing it.

In [ ]:
from medvision_toolkit.learning.patient_profiles import load_sample_patient, summarize_patient

patient_profile = load_sample_patient()
print(summarize_patient(patient_profile))

## 1. Meet Your Environment

An environment is the sterile field for your Python session. In Colab it is refreshed every time you restart the runtime; on your laptop it might live in a virtual environment. Either way, you control the Python version, installed packages, and PATH entries so code behaves predictably.

Run the cell below to report your current interpreter. Capture these details in your lab notes—versions anchor your troubleshooting.

In [ ]:
import platform
import sys
from pathlib import Path

print(f"Python executable: {sys.executable}")
print(f"Python version: {platform.python_version()}")
print(f"Working directory: {Path.cwd()}")

### Vitals Check: Environment Vocabulary

**Which statement best describes a Python environment?**

A. A text file that lists the packages you might install someday.

B. The isolated runtime (interpreter + installed packages) that executes your code.

C. A cloud service that stores all your notebooks.

Think it through before peeking at the answer below.

**Answer:** B. An environment is the interpreter plus its installed packages. Requirements files describe desired state, and storage services host notebooks but do not execute them.

## 2. Prescription for Packages

`pip` is the dispenser that installs specific package versions into your environment. Project MedVision pins dependencies in `requirements.txt` so every learner works with the same toolkit, which keeps clinical exercises reproducible.

Use the cell below to inspect a package that ships with Python. We rely on `importlib.metadata` instead of shell commands so the code works everywhere (Colab, Windows, macOS, Linux).

In [ ]:
from importlib import metadata

PACKAGE_NAME = "pip"

try:
    version = metadata.version(PACKAGE_NAME)
except metadata.PackageNotFoundError as exc:  # type: ignore[attr-defined]
    raise RuntimeError(f"{PACKAGE_NAME} is not installed in this environment.") from exc
else:
    print(f"{PACKAGE_NAME} version: {version}")

### Challenge: Document the Pharmacy Rules

Summarise three safety principles for managing packages in this project (e.g., why we pin versions, when to reinstall, how to verify installs). Replace the `NotImplementedError` with a list of your own sentences.

In [ ]:
from typing import List

package_safety_notes: List[str] | None = None

if package_safety_notes is None:
    raise NotImplementedError("Add three package safety notes before moving on.")

print("Package safety briefing:")
for note in package_safety_notes:
    print(f"- {note}")

In [ ]:

package_safety_notes: List[str] = [
    "Install exactly the versions listed in requirements.txt so everyone sees the same behaviour.",
    "Re-run the setup cell in new Colab sessions instead of mixing manual installs.",
    "Verify critical packages with importlib.metadata.version before running clinical notebooks.",
]

print("Package safety briefing:")
for note in package_safety_notes:
    print(f"- {note}")

## 3. When Alarms Sound

Even the best setups occasionally protest. Keep this mini playbook handy so you can respond quickly and log what worked.

| Symptom | Likely Cause | Action |
| --- | --- | --- |
| `ModuleNotFoundError: medvision_toolkit` | `src/` directory missing from `sys.path` | Re-run the setup cell to append the project path. |
| `pip install` reports SSL or network errors in Colab | Runtime lost internet connection | Restart the runtime, ensure you are authenticated to the network, then rerun setup. |
| Code imports but produces stale behaviour | Old package cache after local update | Run `pip install --upgrade -r requirements.txt` in your virtual environment, then restart the kernel. |

Use the helper below to confirm critical packages without digging into implementation details. If a package is absent, the function raises immediately so you can respond.

In [ ]:
from collections.abc import Iterable
from importlib import metadata


def verify_packages(packages: Iterable[str]) -> None:
    for package in packages:
        try:
            version = metadata.version(package)
        except metadata.PackageNotFoundError as exc:  # type: ignore[attr-defined]
            raise RuntimeError(f"{package} is missing. Re-run the setup cell.") from exc
        else:
            print(f"[OK] {package} {version}")


verify_packages(["pip", "requests"])

### Optional Drill: Install, Inspect, Uninstall

Colab users can safely rehearse the full `pip` cycle on a tiny package. Local users should activate their virtual environment before running the command.

In [ ]:
import subprocess
import sys

PACKAGE = "packaging"

if 'in_colab' in globals() and in_colab:
    print(f"Installing {PACKAGE} for practice…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", PACKAGE])
    subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", PACKAGE], stdout=subprocess.DEVNULL)
    print(f"Completed install/uninstall cycle for {PACKAGE} in Colab runtime.")
else:
    print("Skip this drill if you are unsure which local environment is active. Run it manually once you confirm the correct virtual environment.")

## 4. Sterile Technique Checklist

Before handing the cart to the rest of the team, compile the checks you completed. Use the dataclass below to keep your notes structured. Update the `checklist` so it reflects your actual steps.

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class ChecklistItem:
    description: str
    completed: bool


def render_checklist(items: List[ChecklistItem]) -> None:
    for item in items:
        status = "PASS" if item.completed else "CHECK"
        print(f"[{status}] {item.description}")


checklist: List[ChecklistItem] | None = None

if checklist is None:
    raise NotImplementedError("List the actions you took to certify the environment.")

render_checklist(checklist)

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class ChecklistItem:
    description: str
    completed: bool


def render_checklist(items: List[ChecklistItem]) -> None:
    for item in items:
        status = "PASS" if item.completed else "CHECK"
        print(f"[{status}] {item.description}")


checklist: List[ChecklistItem] = [
    ChecklistItem("Ran the setup cell and confirmed requirements installed", True),
    ChecklistItem("Verified Python version and working directory", True),
    ChecklistItem("Documented package safety practices in lab notes", True),
]

render_checklist(checklist)

## 5. Debrief and Next Steps

Take five minutes to document your findings:

1. **Environment Snapshot:** Record the Python version, key package versions, and date. These notes become your baseline if something changes.
2. **Confidence Check:** On a scale of 1–5, how confident do you feel rerunning this setup on a fresh Colab runtime? What would raise that score?
3. **Escalation Plan:** If the setup cell fails during a high-stakes session, who on your team do you page first and what evidence will you provide?

Once you are satisfied that the cart is sterile and ready, continue to `01_python_first_dose.ipynb`. You now own the environment—keep it predictable, reproducible, and safe for the rest of the course.